In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import warnings
from pathlib import Path
warnings.filterwarnings('ignore')

import os, sys
curruser = os.environ.get('USER')
sys.path.insert(0, './../src/')
sys.path.insert(0, '/home/{}/notebook/support_library/'.format(curruser)) 
sys.path.insert(0, '/home/{}/python35-libs/lib/python3.5/site-packages/'.format(curruser))
# sys.path.insert(0, '/home/ufimtsev1-ys_ca-sbrf-ru/notebooks/labdata/lib/')

import re 
import time
import joblib
import numpy as np
import pandas as pd
import subprocess
# from sshloader import Ssh

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
pd.options.display.max_columns = 1000


# pd.set_option('display.float_format', lambda x: '%.3f' % x)
# pd.set_option('display.max_colwidth', -1)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

from csv import QUOTE_ALL 

from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
# tqdm_notebook.pandas()

import imblearn
import feature_importance

import sklearn
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedShuffleSplit, StratifiedKFold

from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, \
                            classification_report, precision_score, \
                            recall_score, roc_curve, precision_recall_curve, \
                            average_precision_score, make_scorer, confusion_matrix, get_scorer
            
from sklearn.preprocessing import binarize, OneHotEncoder


print(sklearn.__version__)
print(xgboost.__version__)
print(pd.__version__)

Using TensorFlow backend.


0.22.2.post1
0.81
0.25.3


### Train xgboost 

In [3]:
def get_metrics(y_pred, y, average='binary'):
    print(classification_report(y, y_pred, digits=5, ))
    recall = recall_score(y, y_pred, average=average)
    precision=precision_score(y, y_pred,average=average)
    f1=f1_score(y, y_pred,average=average)
    
    return recall, precision, f1

In [4]:
def feature_impotance(model, X_train, count_top_features = 20):
    imp=model.feature_importances_
    names=X_train.columns
    imp, names=map(list, zip(*sorted(zip(imp, names))[::-1][:count_top_features]))
    
    fig, ax = plt.subplots(figsize=(9,12))
    #ax = plt.axes()
    sns.barplot(x=imp, y=names, palette=sns.color_palette('YlGn', 2), ax=ax)
    ax.set_title('Top ' + str(count_top_features) + ' important features')
    plt.show()

## Fetch data for Train

In [5]:
%%time
csvpath = Path.joinpath(Path(os.getcwd()),'csv','data4train_sampled_60pct.csv')
df_part = pd.read_csv(csvpath, encoding='cp1251', sep=';', compression='gzip')

# df_part = pd.read_csv(filepath_or_buffer = csvpath, sep = ';', compression = 'gzip', engine='python',
#                  dtype = {'target' : np.int32, 'OPF_IP': np.float32 ,'OPF_OOO': np.float32 ,'OPF_Other': np.float32 ,'ab_borrowed_capital': np.float32 ,'ab_immobilized_assets': np.float32 ,'ab_losses': np.float32 ,'ab_own_capital': np.float32 ,'acquiring_mobile': np.float32 ,'acquiring_torg': np.float32 ,'acreditiv': np.float32 ,'active_flg': np.float32 ,'all_prod_deals_completed': np.float32 ,'all_prod_deals_created': np.float32 ,'arenda_seyf': np.float32 ,'bank_straxovanie_imush': np.float32 ,'bank_straxovanie_lich': np.float32 ,'bank_straxovanie_otvet': np.float32 ,'branch_cnt': np.float32 ,'campaign_nm': np.float32 ,'cash_management': np.float32 ,'corp_cards': np.float32 ,'corp_cards_prod_deals_completed': np.float32 ,'corp_cards_prod_deals_created': np.float32 ,'corporate_cards': np.float32 ,'cred_application_cnt': np.float32 ,'cred_sdo': np.float32 ,'credit': np.float32 ,'credit_prod_deals_completed': np.float32 ,'credit_prod_deals_created': np.float32 ,'dbo': np.float32 ,'deposit': np.float32 ,'deposit_sertificat': np.float32 ,'deposits': np.float32 ,'depozit_prod_deals_completed': np.float32 ,'depozit_prod_deals_created': np.float32 ,'egr_org_id': np.float32 ,'egrul_org_id': np.float32 ,'einvoicing': np.float32 ,'email': np.float32 ,'factoring': np.float32 , 'fot_balance': np.float32, 'founders_all_cnt': np.float32 ,'founders_fl_cnt': np.float32 ,'founders_foreign_ul_cnt': np.float32 ,'founders_ul_cnt': np.float32 ,'fresh_user': np.float32 ,'garantee_gos': np.float32 ,'garantee_kontract': np.float32 ,'garantee_other': np.float32 ,'gis_bank_payment_flg': np.float32 ,'gis_card_payment_flg': np.float32 ,'gis_cash_payment_flg': np.float32 ,'gis_internet_merchant_flg': np.float32 ,'gis_internet_payment_flg': np.float32 ,'gis_merchant_cnt': np.float32 ,'gis_network_flg': np.float32 ,'gis_opt_merchant_flg': np.float32 ,'gis_production_merchant_flg': np.float32 ,'gis_rosn_merchant_flg': np.float32 ,'industry': np.float32 ,'inkass': np.float32 ,'insure_prod_deals_completed': np.float32 ,'insure_prod_deals_created': np.float32 ,'integrum_lower_bound': np.float32 ,'invest_kredit': np.float32 ,'ip_flg': np.float32 ,'konversion': np.float32 ,'kpp_regions_cnt': np.float32 ,'last_click_all_cnt_m11': np.float32 ,'last_click_all_cnt_m21': np.float32 ,'last_click_all_cnt_m31': np.float32 ,'last_click_day_cnt_m11': np.float32 ,'last_click_day_cnt_m21': np.float32 ,'last_click_day_cnt_m31': np.float32 ,'last_open_all_cnt_m11': np.float32 ,'last_open_all_cnt_m21': np.float32 ,'last_open_all_cnt_m31': np.float32 ,'last_open_day_cnt_m11': np.float32 ,'last_open_day_cnt_m21': np.float32 ,'last_open_day_cnt_m31': np.float32 ,'license_cnt': np.float32 ,'main_inn': np.float32 ,'main_kpp': np.float32 ,'main_organization_id': np.float32 ,'max_zp_empl_cnt': np.float32 ,'merch': np.float32 ,'merch_prod_deals_completed': np.float32 ,'merch_prod_deals_created': np.float32 ,'min_zp_empl_cnt': np.float32 ,'ns_servis': np.float32 ,'ns_terminal': np.float32 ,'oborot_kredit': np.float32 ,'obsluzh_rts_rur': np.float32 ,'obsluzh_rts_val': np.float32 ,'overdraft_kredit': np.float32 ,'products_lizing': np.float32 ,'proekt_finans': np.float32 ,'rko': np.float32 ,'salary': np.float32 ,'salary_prod_deals_completed': np.float32 ,'salary_prod_deals_created': np.float32 ,'samoinkass': np.float32 ,'spec_acc': np.float32 ,'stoplist': np.float32 ,'target': np.float32 ,'tb_Centralno_Chernozemnyj': np.float32 ,'tb_Dalnevostochnyj': np.float32 ,'tb_Moskovskij': np.float32 ,'tb_Povolzhskij': np.float32 ,'tb_Severo_Zapadnyj': np.float32 ,'tb_Uralskij': np.float32 ,'tb_Volgo_Vyatskij': np.float32 ,'total_count_dt': np.float32 ,'total_count_dt_3m': np.float32 ,'total_count_dt_year18': np.float32 ,'total_count_kt': np.float32 ,'total_count_kt_3m': np.float32 ,'total_count_kt_year18': np.float32 ,'ul_org_id': np.float32 ,'valuta_control': np.float32 ,'veksel': np.float32 ,'zarplat_projects': np.float32, 'sum_open_click_camp': np.float32, 'ul_kopf_cd': np.float32 },   
#                  usecols = lambda columns : columns not in ['text', 'text_norm', 'inn', 'kpp', 'organization_id', 'email','campaign_nm', 'idx', 'fresh_user', 'complicity_type', 'ogrn', 'sum_open_click_camp'] 
#                  )

CPU times: user 12min 59s, sys: 1min 6s, total: 14min 5s
Wall time: 14min 5s


### Columns segmentation

In [ ]:
import re
pattern = re.compile(r'.*_m\d{1}1')
pattern.findall('diff_lcl_fo_m11')
ftcols = [col for col in list(df_part.columns) if '_FT_' in col]
respcols = [col for col in list(df_part.columns) if len(pattern.findall(col))!=0]
ml360cols = list(set(list(df_part.columns)) - set(respcols) - set(ftcols))

# Take into account FastText vector

## Train XGBoost: Add Features with FastText Vector

In [ ]:
X_train, X_test, y_train, y_test = \
train_test_split(df_part.drop(['target'], axis=1), df_part.target, 
                 random_state=42, shuffle=True, test_size=0.2)

## XGBoost' Parameters Optimization and FineTuning

In [ ]:
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedShuffleSplit, StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion
from time import time

In [ ]:
classifier = XGBClassifier(random_state=42, n_jobs=40, tree_method='hist',)

### Define pipeline dictionary for optimization with CV 

In [ ]:
neg, pos = df_part.target.value_counts().tolist()

In [ ]:
scale_pos_weight = (neg/pos)
scale_pos_weight

In [ ]:
# Define parameter distribution over which to perform CV

param_dist = {
    "clf__n_estimators":                  [400, 500, 800, 1000],
    "clf__learning_rate":                 [0.005, 0.01, 0.04, 0.08],
    "clf__num_leaves":                    [64, 128, 256, 512],
    "clf__min_data_in_leaf":              [20, 50, 100, 250],
    "clf__colsample_bytree":              [0.5, 0.7, 0.8, 1.],
    "clf__scale_pos_weight":              [2.041, 1.5, 1.2, 1.],
    "clf__max_bin":                       [255, 230, 200, 180],
    "clf__max_depth":                     [5, 6, 8, 10],
    'clf__min_child_samples':             [20, 40, 100],
    'clf__min_child_weight':              [1e-5, 1e-3, 1e-2, 1e-1, 1.],
    'clf__feature_fraction':              [0.7, 0.8, 0.9, 1],
    'clf__bagging_fraction':              [0.2, 0.5, 0.8, 1.],
    'clf__bagging_freq':                  [0, 10, 20, 40],
    'clf__lambda_l1':                     [0, 1e-3, 1e-2, 1e-1],
    'clf__lambda_l2':                     [0, 1e-3, 1e-2, 1e-1],
    'clf__n_jobs':                        [20, 20, 20, 20],
    'clf__tree_method':                   ['hist', 'hist', 'hist', 'hist']
    #'clf__is_unbalance':             [True, False]
}

n_iter_search = 3 # Define number of search iterations
n_folds = 6 # Define number of CV folds

In [ ]:
folds = StratifiedShuffleSplit(n_splits=n_folds, test_size=0.2)

In [ ]:
pipeline = Pipeline([
    ('clf', classifier),
     ])

random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=n_iter_search, cv=folds, 
                                   scoring='recall', n_jobs=8, verbose=2)

In [ ]:
# X_train = df[respcols+ml360cols].drop(['target'], axis=1)
# y_train = df.target

In [ ]:
start = time()

random_search.fit(X_train.astype(dtype=np.float32), y_train)

print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))      
print("\nBest Score = " + str(random_search.best_score_))
print("\nBest Parameters = " + str(random_search.best_params_))

### Fit cls with the best params selected

In [ ]:
classifier = pipeline.set_params(clf__n_estimators                 = random_search.best_params_['clf__n_estimators'],
                                 clf__learning_rate                = random_search.best_params_['clf__learning_rate'],
                                 clf__num_leaves                   = random_search.best_params_['clf__num_leaves'],
                                 clf__max_bin                      = random_search.best_params_['clf__max_bin'],
                                 clf__max_depth                    = random_search.best_params_['clf__max_depth'],
                                 clf__scale_pos_weight             = random_search.best_params_['clf__scale_pos_weight'],
                                 clf__colsample_bytree             = random_search.best_params_['clf__colsample_bytree'],                                 
                                 clf__min_child_samples            = random_search.best_params_['clf__min_child_samples'],
                                 clf__min_data_in_leaf             = random_search.best_params_['clf__min_data_in_leaf'],
                                 clf__min_child_weight             = random_search.best_params_['clf__min_child_weight'],
                                 clf__feature_fraction             = random_search.best_params_['clf__feature_fraction'],
                                 clf__bagging_fraction             = random_search.best_params_['clf__bagging_fraction'],
                                 clf__bagging_freq                 = random_search.best_params_['clf__bagging_freq'],
                                 clf__lambda_l1                    = random_search.best_params_['clf__lambda_l1'],
                                 clf__lambda_l2                    = random_search.best_params_['clf__lambda_l2'],
                                 clf__n_jobs                       = 40,
                                 clf__tree_method                  = random_search.best_params_['clf__tree_method']                                 
                                 )

In [ ]:
classifier.fit(X_train.astype(dtype=np.float32), y_train)

In [ ]:
y_pred = classifier.predict(X_test.astype(dtype=np.float32))

recall_ts, precision_ts, f1_ts =  get_metrics(y_pred.astype(int), y_test.astype(int))
                                                  
print('Hold Out: ')
print('Precision: %7.5f, Recall: %7.5f, F_1: %7.5f' % 
(precision_ts, recall_ts, f1_ts))

In [ ]:
joblib.dump(classifier, './pkl/xgb_cls_tuned_pipe_wFT_200606.pkl')

## LightGBM' Parameters Optimization and FineTuning

In [ ]:
X_train.shape

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
lgm=LGBMClassifier(seed=42, n_jobs = 20)

In [ ]:
folds = StratifiedShuffleSplit(n_splits=n_folds, test_size=0.2)

In [ ]:
pipeline = Pipeline([
    ('clf', lgm),
     ])

random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=n_iter_search, cv=folds, 
                                   scoring='recall', n_jobs=8, verbose=2)

In [ ]:
start = time()

random_search.fit(X_train.astype(dtype=np.float32), y_train)

print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))      
print("\nBest Score = " + str(random_search.best_score_))
print("\nBest Parameters = " + str(random_search.best_params_))

### Fit cls with the best params selected

In [ ]:
lgm    = pipeline.set_params(clf__n_estimators                 = random_search.best_params_['clf__n_estimators'],
                             clf__learning_rate                = random_search.best_params_['clf__learning_rate'],
                             clf__num_leaves                   = random_search.best_params_['clf__num_leaves'],
                             clf__max_bin                      = random_search.best_params_['clf__max_bin'],
                             clf__max_depth                    = random_search.best_params_['clf__max_depth'],
                             clf__scale_pos_weight             = random_search.best_params_['clf__scale_pos_weight'],
                             clf__colsample_bytree             = random_search.best_params_['clf__colsample_bytree'],                                 
                             clf__min_child_samples            = random_search.best_params_['clf__min_child_samples'],
                             clf__min_data_in_leaf             = random_search.best_params_['clf__min_data_in_leaf'],
                             clf__min_child_weight             = random_search.best_params_['clf__min_child_weight'],
                             clf__feature_fraction             = random_search.best_params_['clf__feature_fraction'],
                             clf__bagging_fraction             = random_search.best_params_['clf__bagging_fraction'],
                             clf__bagging_freq                 = random_search.best_params_['clf__bagging_freq'],
                             clf__lambda_l1                    = random_search.best_params_['clf__lambda_l1'],
                             clf__lambda_l2                    = random_search.best_params_['clf__lambda_l2'],
                             clf__n_jobs                       = 40,
                             clf__tree_method                  = random_search.best_params_['clf__tree_method']                            
                             )

In [ ]:
lgm.fit(X_train.astype(float), y_train)

In [ ]:
y_pred = lgm.predict(X_test.astype(dtype=np.float32))

recall_ts, precision_ts, f1_ts =  get_metrics(y_pred.astype(int), y_test.astype(int))
                                                  
print('Hold Out: ')
print('Precision: %7.5f, Recall: %7.5f, F_1: %7.5f' % 
(precision_ts, recall_ts, f1_ts))

In [ ]:
joblib.dump(lgm, './pkl/lgbm_cls_tuned_pipe_wFT_200606.pkl')

## RandomForestClassifier' Parameters Optimization and FineTuning

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.impute import SimpleImputer

In [ ]:
rfcls = RandomForestClassifier()

In [ ]:
X_train.shape

In [ ]:
imp=SimpleImputer(missing_values=np.nan, strategy='mean')
X_train_meta=pd.DataFrame(imp.fit_transform(X_train), columns=X_train.columns)

In [ ]:
# Define parameter distribution over which to perform CV

param_dist = {
    "clf__n_estimators":                  [400, 500, 800, 1000],
    "clf__max_leaf_nodes":                [128, 256, 512, 1024],
    "clf__min_weight_fraction_leaf":      [0, 0.05, 0.1, 0.4],
    "clf__max_depth":                     [6, 8, 10, 14],
    'clf__max_features':                  [0.7, 0.8, 0.9, 1],
#     'clf__max_samples':                   [0.7, 0.8, 0.9, 1],
    'clf__min_impurity_decrease':         [1.e-9, 1.e-7, 1.e-5, 0.0],
    'clf__n_jobs':                        [20, 20, 20, 20],
    'clf__class_weight':                  [{0:1,1:1},{0:1,1:2},{0:1,1:4},{0:1,1:6}],
    'clf__bootstrap':                     ['true', 'true', 'true', 'true'],
    'clf__criterion':                     ['gini', 'gini', 'gini', 'gini']
    #'clf__is_unbalance':             [True, False]
}

n_iter_search = 3 # Define number of search iterations
n_folds = 6 # Define number of CV folds

In [ ]:
folds = StratifiedShuffleSplit(n_splits=n_folds, test_size=0.2)

In [ ]:
pipeline = Pipeline([
    ('clf', rfcls),
     ])

random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=n_iter_search, cv=folds, 
                                   scoring='recall', n_jobs=8, verbose=2)

In [ ]:
start = time()

random_search.fit(X_train_meta.astype(dtype=np.float32), y_train)

print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))      
print("\nBest Score = " + str(random_search.best_score_))
print("\nBest Parameters = " + str(random_search.best_params_))

### Fit cls with the best params selected

In [ ]:
rfcls = RandomForestClassifier()
pipeline = Pipeline([
    ('clf', rfcls),
     ])
rfcls  = pipeline.set_params(clf__n_estimators                 = 800,
#                              clf__max_leaf_nodes               = 512,
#                              clf__min_weight_fraction_leaf     = 1e-4,
                             clf__max_depth                    = 6,
#                              clf__max_features                 = 0.9,
                             #clf__min_impurity_decrease        = 1e-5,
                             #clf__class_weight                 = {0:1,1:3},                                 
                             clf__bootstrap                    = True,
                             clf__n_jobs                       = 50,
                             clf__criterion                    = 'gini'
                             )

In [ ]:
rfcls

In [ ]:
# rfcls  = pipeline.set_params(clf__n_estimators                 = random_search.best_params_['clf__n_estimators'],
#                              clf__max_leaf_nodes               = random_search.best_params_['clf__max_leaf_nodes'],
#                              clf__min_weight_fraction_leaf     = random_search.best_params_['clf__min_weight_fraction_leaf'],
#                              clf__max_depth                    = random_search.best_params_['clf__max_depth'],
#                              clf__max_features                 = random_search.best_params_['clf__max_features'],
#                              clf__min_impurity_decrease        = random_search.best_params_['clf__min_impurity_decrease'],
#                              clf__class_weight                 = random_search.best_params_['clf__class_weight'],                                 
#                              clf__bootstrap                    = random_search.best_params_['clf__bootstrap'],
#                              clf__n_jobs                       = 40
#                              clf__criterion                    = 'gini'
#                              )

In [ ]:
rfcls.fit(X_train_meta.astype(dtype=np.float32), y_train)

In [ ]:
X_test_meta=pd.DataFrame(imp.transform(X_test), columns=X_test.columns)
y_pred = rfcls.predict(X_test_meta)

recall_ts, precision_ts, f1_ts =  get_metrics(y_pred.astype(int), y_test.astype(int))
                                                  
print('Hold Out: ')
print('Precision: %7.5f, Recall: %7.5f, F_1: %7.5f' % 
(precision_ts, recall_ts, f1_ts))

In [ ]:
del X_test_meta, X_train_meta

# Imbalanced Learn

In [6]:
from collections import Counter
from imblearn import over_sampling
from sklearn.impute import SimpleImputer

from xgboost import XGBClassifier

from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, \
                            classification_report, precision_score, \
                            recall_score, roc_curve, precision_recall_curve, \
                            average_precision_score, make_scorer, confusion_matrix, get_scorer

from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedShuffleSplit, StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion
from time import time

In [7]:
zero_std_cols = df_part.columns[df_part.std(axis=0) == 0.].values.tolist()
df_part.drop(columns=zero_std_cols, axis=1, inplace=True)
print('Removed {} constant columns'.format(len(zero_std_cols)))

Removed 76 constant columns


In [8]:
imp=SimpleImputer(missing_values=np.nan, strategy='mean')
df_part_meta=pd.DataFrame(imp.fit_transform(df_part.astype(dtype=np.float32).values), columns=df_part.columns)

In [9]:
X_train, X_test, y_train, y_test = \
train_test_split(df_part_meta.drop(['target'], axis=1), df_part_meta.target, 
                 random_state=42, shuffle=True, test_size=0.1)

In [23]:
Counter(df_part.target)

Counter({0.0: 4612709, 1.0: 2259592})

In [19]:
n, p = df_part.target.value_counts().tolist()
p/n

0.48986224797618927

In [13]:
adasyn = over_sampling.BorderlineSMOTE(sampling_strategy=0.6, 
                                   random_state=np.random.RandomState(42), 
                                   k_neighbors=5,
                                   n_jobs=-1)

In [ ]:
X_train_res, y_train_res = adasyn.fit_resample(X_train, y_train)

In [ ]:
cls_pipe = joblib.load('./pkl/xgb_cls_tuned_pipe_wFT_200606.pkl')
xgb = cls_pipe['clf']
xgb_params = xgb.get_params()
classifier = XGBClassifier(**xgb_params)
classifier

In [ ]:
classifier.fit(X_train_res.astype(dtype=np.float32), y_train_res)

In [ ]:
y_pred = classifier.predict(X_test.astype(dtype=np.float32))

recall_ts, precision_ts, f1_ts =  get_metrics(y_pred.astype(int), y_test.astype(int))
                                                  
print('Hold Out: ')
print('Precision: %7.5f, Recall: %7.5f, F_1: %7.5f' % 
(precision_ts, recall_ts, f1_ts))